In [1]:
# -*- coding: utf-8 -*-

""" Auto Encoder Example.
Using an auto encoder on MNIST handwritten digits.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
"""
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import *

# Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

df = pd.read_csv("../dat/diabetic.txt", header=None);
DATA_NAME = 'diabetic'
DATA_PATH = "../dat/diabetic.txt"
CATEGORICAL_F = {'diabetic':[0,1,18]} #18
TRAINING_RATIO = 0.9 #0.2
MISS_IMP_PROB = 0.3
feature_cat = CATEGORICAL_F[DATA_NAME] 

In [10]:
#Load data and get numerical feature
X,Y,feature_numerical = load_data(DATA_PATH, DATA_NAME,feature_cat)

#Split data into train and test. This is temporary, cross-validation should be adopted later
# x_train, x_test = X[:int(len(X) * TRAINING_RATIO)], X[int(len(X) * TRAINING_RATIO):]
# y_train, y_test = Y[:int(len(X) * TRAINING_RATIO)], Y[int(len(X) * TRAINING_RATIO):]
x_train, x_test = X[:int(len(X) * TRAINING_RATIO)], X[:int(len(X) * TRAINING_RATIO)]
y_train, y_test = Y[:int(len(X) * TRAINING_RATIO)], Y[:int(len(X) * TRAINING_RATIO)]
x_test = x_test.reset_index(drop=True)
x_oracle = x_test.copy()


#Preprocessing training data (gaussian normalization only)
x_train = preprocessing_train(x_train,feature_numerical, feature_cat)
x_oracle = preprocessing_train(x_oracle, feature_numerical, feature_cat)

#The following step is used to create missing value in the testing data
#The missing strategy is: The more important the the feature, the higher the prob it miss

#The importance of feature is calculated by random forest(i.e. feature_importance_)
feature_imp, feature_notimp = Calculate_feature_imp(x_train,y_train)

#We create anotehr matrix missing_mask, which denotes the missing entry. 1 for non-missing and 0 for missing
x_test,missing_mask = preprocessing_test(x_test,feature_imp,feature_cat,feature_numerical,MISS_IMP_PROB)
missing_mask = MissingMaskCreation(missing_mask,feature_numerical)

# #Clean the column feature end with string 'nan'
x_test = CleanCatNan(x_test,feature_cat)
missing_mask = CleanCatNan(missing_mask,feature_cat)

# # Only consider the common feature seen in both training and testing data
x_train, x_test,unseen = RemoveUnseenFeature(x_train, x_test)

for un in unseen:
    if un in missing_mask.columns:
        missing_mask.drop(un,axis=1,inplace=True)
        x_oracle.drop(un,axis=1,inplace=True)

# #Scale the testing data, only scale data based on seen entries
x_test = Normalization(x_test, feature_numerical)
x_test = np.nan_to_num(x_test)
x_train = x_train.values
y_train = y_train.values
y_test = y_test.values

Unseen feature in test: 
[]


In [11]:
TMP = len(x_train)

In [13]:
# Parameters
learning_rate = 0.1
training_epochs = 3
# batch_size = 256
batch_size = 19
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 19 # 1st layer num features
n_hidden_2 = 100 # 2nd layer num features
n_input = TMP # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

mask = tf.placeholder(tf.float32, [None, 20]) 

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(n_input / batch_size) #(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = x_train.T, y_train.T # mnist.train.next_batch(batch_size) # 256x1, 256x1
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))

    print("Optimization Finished!")


#     numpy_arr = test_x.as_matrix()
#     numpy_missing_mask = missing_mask.as_matrix()
#     train = tf.placeholder(tf.bool, [])
    sess.run(y_pred, feed_dict={X:x_test.T})
#     # Applying encode and decode over test set
#     encode_decode = sess.run(
#         y_pred, feed_dict={X:numpy_arr,mask:numpy_missing_mask,train:False}) #{X: mnist.test.images[:examples_to_show]})
#     # Compare original images with their reconstructions
#     f, a = plt.subplots(2, 10, figsize=(10, 2))
#     for i in range(examples_to_show):
#         a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
#         a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
#     f.show()
#     plt.draw()
#     plt.waitforbuttonpress()

Epoch: 0001 cost= 1.084210873
Epoch: 0002 cost= 0.966805220
Epoch: 0003 cost= 0.676048636
Optimization Finished!


In [14]:
loss = tf.reduce_mean(tf.pow(y_pred - y_test, 2))

In [23]:
loss.eval(feed_dict={X: x_test.T})

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`